# Ανάκτησης Πληροφορίας - Δημιουργία Μηχανής Αναζήτησης
- Μπηλιώνη Παρασκευή <br> Α.Μ. ice20390286
  
- Πλάγου Αικατερίνη  <br> Α.Μ. ice20390191

# 1. Συλλογή δεδομένων

Εισαγωγή και αρχικοποίηση των απαραίτητων βιβλιοθηκών.

In [1]:
# Import libraries
from bs4 import BeautifulSoup  # Web scraping and parsing 
import requests                # Makes HTTP request
import json                    # Handles JSON data
import nltk                    # Text processing
import string                  # Handles strings
import sys                     # System-specific parameters and functions
from nltk.corpus import stopwords       # Handles stopwords in text processing
from nltk.stem import WordNetLemmatizer # Word lemmatization
from collections import defaultdict     # Creates dictionaries 
import ipywidgets as widgets            # Creates widgets
import numpy as np                      # Does computations
from IPython.display import display, Markdown                # Displays widgets and text
from sklearn.feature_extraction.text import TfidfVectorizer  # TF-IDF vectorization
from sklearn.metrics.pairwise import cosine_similarity       # Calculates vector similarity (used in VSM)
from rank_bm25 import BM25Okapi                              # Rankins documents (used in okapi BM25)
from sklearn.metrics import precision_score, recall_score, f1_score  # Calculates engine evaluation metrics
from IPython.display import display, Markdown               
import pandas as pd   

# Download NLTK datasets
# Tokenizer models
nltk.download('punkt')  
# List of stopwords
nltk.download('stopwords')  
# For lemmatization
nltk.download('wordnet')    

# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words('english'))  # Set of stopwords
lemmatizer = WordNetLemmatizer()              # Lemmatizer for reducing words to base forms


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vivh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vivh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vivh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Επιλογή πηγής από την οποία θα ξεκινήσει η συλλογή εγγράφων. 

In [2]:
# Starting wikipedia link to scrape
wiki_url = 'https://en.wikipedia.org/wiki/Data_analysis'

try:
    # Send request to starting link
    response = requests.get(wiki_url)
    # Raise an exception for errors
    response.raise_for_status()
        
    # Parse HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
        
# Handle any exceptions during the request
except requests.RequestException as e:
    print(f"____Request failed____\n{e}\n")

display(Markdown(f"Data will be scraped from this starting page: [{wiki_url}]({wiki_url})<br><br>"))


Data will be scraped from this starting page: [https://en.wikipedia.org/wiki/Data_analysis](https://en.wikipedia.org/wiki/Data_analysis)<br><br>

Αποθήκευση δεδομένων σε JSON αρχείο.
Σε περίπτωση σφάλματος κατά την διάρκεια της διαδικασίας, εμφανίζεται κατάλληλο μήνυμα λάθους.

In [3]:
def store_things(things, filename):
    try:
        # Open the specified file in write mode
        with open(filename, 'w') as file:
            # Convert data to a JSON string and write it to the file
            json.dump(things, file, indent = 4)

    # If the process fails print error message
    except Exception as e:
        print(f"____Error saving____\n{e}")
        

Απόκτηση εγγράφων/συνδέσμων από την κύρια σελίδα.

In [4]:
def get_links(soup):
    # Base wikipedia url
    https = 'https://en.wikipedia.org'  
    # For storing valid links
    links = []  

    display(Markdown("Links saved: <br>"))
    # Find all anchor tags with 'href' attribute
    for link in soup.find_all('a', href = True):
        # Extract link reference
        url = link.get('href') 

        # Check if the link is a wikipedia aticle and filter out irrelevant links
        if url.startswith('/wiki/') and not any(
            url.startswith(f'/wiki/{keyword}')
            for keyword in ['Wikipedia', 'Help', 'Special', 'Portal', 'Talk', 'Category', 'File', 'Main_Page']):
            # Construct full wikipedia url
            full_url = f"{https}{url}"

            # Skip links that appear if they are already saved
            if full_url not in links:
                links.append(full_url)
                #display(Markdown(f"[{full_url}]({full_url})<br>"))
                print(f"{full_url}\n")
                
        # Collect 70 first valid links
        if len(links) >= 70:
            break 
            
    # Return links
    return links  


# Collect and store links from the main page
links = get_links(soup)
store_things(links, 'wikipedia_collected_urls.json')

# For tracking visited pages
visited_links = set()

# Mark the starting link as visited
visited_links.add(wiki_url)


Links saved: <br>

https://en.wikipedia.org/wiki/Data_analysis

https://en.wikipedia.org/wiki/Statistics

https://en.wikipedia.org/wiki/Data_and_information_visualization

https://en.wikipedia.org/wiki/Exploratory_data_analysis

https://en.wikipedia.org/wiki/Information_design

https://en.wikipedia.org/wiki/Interactive_data_visualization

https://en.wikipedia.org/wiki/Descriptive_statistics

https://en.wikipedia.org/wiki/Statistical_inference

https://en.wikipedia.org/wiki/Statistical_graphics

https://en.wikipedia.org/wiki/Plot_(graphics)

https://en.wikipedia.org/wiki/Infographic

https://en.wikipedia.org/wiki/Data_science

https://en.wikipedia.org/wiki/Tamara_Munzner

https://en.wikipedia.org/wiki/Ben_Shneiderman

https://en.wikipedia.org/wiki/John_Tukey

https://en.wikipedia.org/wiki/Edward_Tufte

https://en.wikipedia.org/wiki/Simon_Wardley

https://en.wikipedia.org/wiki/Hans_Rosling

https://en.wikipedia.org/wiki/David_McCandless

https://en.wikipedia.org/wiki/Kim_Albrecht

https://en.wikipedia.org/

Συλλογή παραγράφων από κάθε σελίδα. 

In [5]:
def get_paragraphs(soup, visited_paragraphs, link):
    # Stores paragraphs
    paragraphs = []  

    # Remove the content from superscripts and references
    for sup in soup.find_all(['sup', 'reflist']):
        sup.decompose()  

    # Find all paragraph tags 
    for p in soup.find_all('p'):
        # Extract text from each paragraph
        text = p.get_text()  
        
        # Remove empty paragraphs and those containing the term 'displaystyle' to avoid mathematical functions
        if text and 'displaystyle' not in text.lower():
            # Calculate the number of words in the paragraph
            word_count = len(text.split())  
            
            # Include paragraphs with word count between 50 and 100 and avoid duplicates
            if 50 <= word_count <= 100 and text not in visited_paragraphs:
                # Store paragraph with source link
                paragraphs.append({'text': text, 'link': link}) 
                # Mark the paragraph as visited to avoid repetition
                visited_paragraphs.add(text)  
                        
    # Return filtered paragraphs         
    return paragraphs  


# For tracking visited paragraphs
visited_paragraphs = set()

# Collect paragraphs from starting page
original_paragraphs = get_paragraphs(soup, visited_paragraphs, wiki_url)

display(Markdown("Paragraphs saved from starting page: <br>"))
for paragraph in original_paragraphs:
    print(f"{paragraph['text']}")


Paragraphs saved from starting page: <br>

Data analysis is the process of inspecting, cleansing, transforming, and modeling data with the goal of discovering useful information, informing conclusions, and supporting decision-making. Data analysis has multiple facets and approaches, encompassing diverse techniques under a variety of names, and is used in different business, science, and social science domains. In today's business world, data analysis plays a role in making decisions more scientific and helping businesses operate more effectively.

The data is necessary as inputs to the analysis, which is specified based upon the requirements of those directing the analytics (or customers, who will use the finished product of the analysis). The general type of entity upon which the data will be collected is referred to as an experimental unit (e.g., a person or population of people). Specific variables regarding a population (e.g., age and income) may be specified and obtained.  Data may be numerical or categorical (i.e., a text

# 2. Προεπεξεργασία κειμένου (Text Procissing)

Διαμόρφωση κειμένου μετά από αφαίρεση διακόπτουσων λέξεων, ειδικών χαρακτήρων και λημματοποίση όρων. 

In [6]:
def clean_text(text):
    # Tokenize text into individual words and convert to lowercase for better search results
    tokens = nltk.word_tokenize(text.lower())

    # List to store cleaned tokens
    cleaned_tokens = []

    # Remove punctuation and non alphanumeric tokens
    for token in tokens:
        if token not in string.punctuation and token.isalnum():
            cleaned_tokens.append(token)

    # List to store tokens after stopword removal
    filtered_tokens = []

    # Remove stopwords from the tokenized text
    for token in cleaned_tokens:
        if token not in stop_words:
            filtered_tokens.append(token)

    # Initialize a list to store lemmatized tokens
    lemmatized_tokens = []

    # Lemmatize the tokens
    for token in filtered_tokens:
        lemmatized_tokens.append(lemmatizer.lemmatize(token))

    # Return processed tokens
    return lemmatized_tokens

cleaned_paragraphs = []

# Clean the collected paragraphs using text preprocessing
for paragraph in original_paragraphs:
    #clean_paragraph = ' '.join(clean_text(paragraph))
    clean_paragraph = clean_text(paragraph['text'])
    cleaned_paragraphs.append({'tokens': clean_paragraph, 'link': paragraph['link']})

display(Markdown("Processed paragraphs from starting page: <br>"))
for paragraph in cleaned_paragraphs:
    print(f"{paragraph['tokens']}\n")
    

Processed paragraphs from starting page: <br>

['data', 'analysis', 'process', 'inspecting', 'cleansing', 'transforming', 'modeling', 'data', 'goal', 'discovering', 'useful', 'information', 'informing', 'conclusion', 'supporting', 'data', 'analysis', 'multiple', 'facet', 'approach', 'encompassing', 'diverse', 'technique', 'variety', 'name', 'used', 'different', 'business', 'science', 'social', 'science', 'domain', 'today', 'business', 'world', 'data', 'analysis', 'play', 'role', 'making', 'decision', 'scientific', 'helping', 'business', 'operate', 'effectively']

['data', 'necessary', 'input', 'analysis', 'specified', 'based', 'upon', 'requirement', 'directing', 'analytics', 'customer', 'use', 'finished', 'product', 'analysis', 'general', 'type', 'entity', 'upon', 'data', 'collected', 'referred', 'experimental', 'unit', 'person', 'population', 'people', 'specific', 'variable', 'regarding', 'population', 'age', 'income', 'may', 'specified', 'obtained', 'data', 'may', 'numerical', 'categorical', 'text', 'label', 'number']

['mathemat

Ληψη και εξαγωγή παραγράφων από κάθε σύνδεσμο. 
Λαμβάνεται το περιεχόμενο από έναν σύνδεσμο και εξάγονται οι παράγραφοι, σε περίπτωση που δεν έχουν ήδη καταχωρηθεί.

In [7]:
def paragraphs_within_links(link, visited_links, visited_paragraphs):
    # Skip the link and return an empty list if it has already been processed
    if link in visited_links:
        return []

    try:
        # Send get request to link and raise error if the request was unsuccessful
        response = requests.get(link)
        response.raise_for_status()
        
        # Parse the pages content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Mark current link as visited
        visited_links.add(link)
        
        # Extract and return paragraphs from the page
        return get_paragraphs(soup, visited_paragraphs, link)
        
    # Exception for request errors
    except requests.RequestException as e:
        print(f"Error retrieving links: {e}")
        # Return an empty list if there is an error
        return []  

display(Markdown("Scraping paragraphs from each link <br>"))
progress = widgets.IntProgress(
    value = 0,
    min = 0,
    max = len(links)
)

display(progress)

# Get paragraphs from each link and avoid re visiting links
for i, link in enumerate(links):
    # Get paragraphs from current link
    link_paragraphs = paragraphs_within_links(link, visited_links, visited_paragraphs)

    # Extend list of original paragraphs adding the new ones
    original_paragraphs.extend(link_paragraphs)

    # Clean and store paragraphs from current link
    for paragraph in link_paragraphs:
        #clean_paragraph = ' '.join(clean_text(paragraph))
        clean_paragraph = clean_text(paragraph['text'])
        cleaned_paragraphs.append({'tokens': clean_paragraph, 'link': paragraph['link']})

    # Update progress bar
    progress.value = i + 1


Scraping paragraphs from each link <br>

IntProgress(value=0, max=70)

Αποθήκευση επιλεγμένων παραγράφων στην αρχική και επεξεργασμένη μορφή τους.

In [8]:
store_things(original_paragraphs, 'wikipedia_paragraphs.json')
store_things(cleaned_paragraphs, 'wikipedia_paragraphs_cleaned.json')

display(Markdown("How some of the paragraphs are saved: <br>"))
for paragraph in original_paragraphs[:10]:
    text = paragraph['text']
    link = paragraph['link']

    print(f"text: {text}")
    print(f"link: {link}\n\n")
    

How some of the paragraphs are saved: <br>

text: Data analysis is the process of inspecting, cleansing, transforming, and modeling data with the goal of discovering useful information, informing conclusions, and supporting decision-making. Data analysis has multiple facets and approaches, encompassing diverse techniques under a variety of names, and is used in different business, science, and social science domains. In today's business world, data analysis plays a role in making decisions more scientific and helping businesses operate more effectively.

link: https://en.wikipedia.org/wiki/Data_analysis


text: The data is necessary as inputs to the analysis, which is specified based upon the requirements of those directing the analytics (or customers, who will use the finished product of the analysis). The general type of entity upon which the data will be collected is referred to as an experimental unit (e.g., a person or population of people). Specific variables regarding a population (e.g., age and income) may be specified a

In [9]:
display(Markdown("How some of the processed and tokenised paragraphs are saved: <br>"))
for paragraph in cleaned_paragraphs[:10]:
    tokens = paragraph['tokens']
    link = paragraph['link']

    print(f"tokens: {tokens}\n")
    print(f"link: {link}\n\n")
    

How some of the processed and tokenised paragraphs are saved: <br>

tokens: ['data', 'analysis', 'process', 'inspecting', 'cleansing', 'transforming', 'modeling', 'data', 'goal', 'discovering', 'useful', 'information', 'informing', 'conclusion', 'supporting', 'data', 'analysis', 'multiple', 'facet', 'approach', 'encompassing', 'diverse', 'technique', 'variety', 'name', 'used', 'different', 'business', 'science', 'social', 'science', 'domain', 'today', 'business', 'world', 'data', 'analysis', 'play', 'role', 'making', 'decision', 'scientific', 'helping', 'business', 'operate', 'effectively']

link: https://en.wikipedia.org/wiki/Data_analysis


tokens: ['data', 'necessary', 'input', 'analysis', 'specified', 'based', 'upon', 'requirement', 'directing', 'analytics', 'customer', 'use', 'finished', 'product', 'analysis', 'general', 'type', 'entity', 'upon', 'data', 'collected', 'referred', 'experimental', 'unit', 'person', 'population', 'people', 'specific', 'variable', 'regarding', 'population', 'age', 'income', 'may', 'specified', 'obtained', 'data', 'may'

# 3. Ευρετήριο (Indexing)

Δημιουργία ανεστραμμένου ευρετήριου από τις επιλεγμένες και επεξεργασμένες παραγράφους και αποθήκευσή του σε αρχείο JSON.

In [10]:
def build_inverted_index(cleaned_paragraphs):
    inverted_index = defaultdict(set)
    
    # Look through each tokenized paragraph and assign a unique ID
    for paragraph_id, paragraph in enumerate(cleaned_paragraphs):
        # Get the tokens from each paragraph
        tokens = paragraph['tokens']
        
        # Add each token to the inverted index with its paragraph ID
        for token in tokens:
            inverted_index[token].add(paragraph_id)
             
    return inverted_index


def store_inverted_index(inverted_index):
    try:
        # Convert sets to lists to save in JSON file
        serializable_index = {}
        for term, paragraph_ids in inverted_index.items():
            serializable_index[term] = list(paragraph_ids)

        # Save the converted index to the file
        with open('inverted_index.json', 'w') as file:
            json.dump(serializable_index, file, indent = 4)

    except Exception as e:
        print(f"____Error saving inverted index____\n{e}")

# Build inverted index from cleaned paragraphs
inverted_index = build_inverted_index(cleaned_paragraphs)

# Store generated inverted index
store_inverted_index(inverted_index)


# Convert the inverted index into a list of token and paragraph IDs
index = list(inverted_index.items())
# Create a data frame from the inverted index
inverted_df = pd.DataFrame(index, columns = ['Token', 'Paragraph ID'])
# Display the inverted index
display(Markdown("Inverted Index <br>"))
pd.set_option('display.max_colwidth', None)
display(inverted_df)


Inverted Index <br>

,Token,Paragraph ID
0,data,"{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 532, 31, 38, 43, 44, 48, 49, 50, 51, 52, 54, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 70, 74, 75, 77, 79, 80, 81, 82, 83, 86, 87, 88, 89, 526, 92, 93, 95, 97, 101, 102, 108, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 134, 135, 136, 142, 143, 144, 145, 146, 149, 150, 151, 159, 161, 163, ...}"
1,analysis,"{0, 1, 514, 5, 7, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 528, 529, 27, 35, 36, 40, 44, 59, 61, 63, 65, 66, 67, 75, 77, 78, 82, 92, 113, 116, 120, 121, 122, 123, 124, 127, 134, 135, 137, 142, 143, 145, 146, 152, 168, 172, 173, 178, 180, 203, 212, 255, 297, 299, 304, 318, 319, 323, 340, 350, 360, 363, 387, 400, 401, 410, 429, 432, 435, 449, 454, 472, 475, 487, 510}"
2,process,"{0, 512, 132, 5, 516, 265, 12, 13, 268, 269, 270, 271, 273, 275, 276, 21, 277, 23, 278, 279, 26, 27, 280, 282, 31, 419, 429, 56, 326, 333, 80, 339, 468, 87, 347, 350, 479, 480, 481, 401, 486, 402, 377}"
3,inspecting,{0}
4,cleansing,{0}
...,...,...
4958,simulated,{532}
4959,instability,{532}
4960,implicit,{533}
4961,demanding,{533}


# 4. Μηχανή αναζήτησης (Search Engine) 

α) Επεξεργασία ερωτήματος (Query Processing)

Μετατροπή ερωτήματος από infix σε postfix μορφή και ό,τι δεν συμπεριλαμβάνεται στην άλγεβρα Boole, γίνεται lemmatized. 

In [11]:
def infix_to_postfix(query):
    # Define operator precedence (higher value means higher precedence)
    precedence = {'NOT': 3, 'AND': 2, 'OR': 1}
    
    # Output and operator stack
    output = []  
    operators = []

    # Split the query into tokens
    tokens = query.split()
    
    # Process each token
    for token in tokens:
        # If the token is an operator handle based on precedence
        if token in precedence:
            # Pop operators with higher or equal precedence from the stack
            while operators and precedence.get(operators[-1], 0) >= precedence[token]:
                output.append(operators.pop())
            # Push the current operator to the stack
            operators.append(token)  

        # If the token is left parenthesis push it onto the stack
        elif token == '(':
            operators.append(token)

        # If the token is right parenthesis pop until the matching left parenthesis
        elif token == ')':
            while operators and operators[-1] != '(':
                output.append(operators.pop())
            # Remove left parenthesis from stack
            operators.pop()  

        # If the token is a word or other charachters 
        else:
            # Lemmatize and turn into lowecase
            token = lemmatizer.lemmatize(token.lower())
            # Rremove word or number charachters
            if token.isalnum():
                output.append(token)

    # Pop any remaining operators from the stack and append them to output
    while operators:
        output.append(operators.pop())

    # Return the query in postfix notation
    return output


display(Markdown("Example of how queries are processed <br>"))
test_query = "( data AND analysis ) OR NOT science"
# Convert the query from infix to postfix notation
test_postfix_query = infix_to_postfix(test_query)

# Print the original query and its postfix conversion
print(f"Original queryn (Infix):   {test_query}")
print(f"Converted query (Postfix): {(test_postfix_query)}\n")


Example of how queries are processed <br>

Original queryn (Infix):   ( data AND analysis ) OR NOT science
Converted query (Postfix): ['data', 'analysis', 'AND', 'science', 'NOT', 'OR']



Συλλογή παραγράφων με βάση το ήδη επεξεργασμένο ερώτημα. 

In [12]:
def evaluate_postfix(postfix_tokens, inverted_index, num_paragraphs):
    # Evaluating the postfix expression
    stack = []  
    
    # Handling NOT operations
    all_paragraphs = set(range(num_paragraphs))  

    # Look through each token in the expression and preform nessesary operations
    for token in postfix_tokens:
        if token == 'AND':
            # Pop the top two sets
            right = stack.pop()  
            left = stack.pop()
            # Push the result of addition to stack
            stack.append(left & right)  

        elif token == 'OR':
            right = stack.pop()
            left = stack.pop()
            # Push the result of union to stack
            stack.append(left | right)  

        # If the token is NOT operator calculate the difference 
        elif token == 'NOT':
            operand = stack.pop()
            # Push ducuments that are not in list to stack
            stack.append(all_paragraphs - operand)  

        # If the token is search term retrieve the matching paragraph IDs from inverted index
        else:
            # Push matching paragraph IDs to stack
            stack.append(inverted_index.get(token, set())) 

    # Return the matching paragraph IDs if there are any or empty set
    if stack:
        return stack.pop()
    else:
        return set()


test_matching_paragraphs = evaluate_postfix(test_postfix_query, inverted_index, len(cleaned_paragraphs))
display(Markdown("Matching paragraph IDs for example query:"))
print(f"{test_matching_paragraphs}")


Matching paragraph IDs for example query:

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 42, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 116, 117, 120, 121, 122, 123, 124, 126, 127, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 158, 159, 160, 161, 162, 164, 166, 168, 169, 171, 172, 173, 176, 177, 178, 180, 181, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244

<br>β) Κατάταξη αποτελεσμάτων (Ranking)

Δημιουργία πίνακα TF-IDF με βάση τα αποτελέσματα του ερωτήματος. 

In [13]:
def tf_idf(results, cleaned_paragraphs):
    # Return nothing if there are no results
    if not results:
        return None, [], [], None

    # Store the filtered paragraphs and their IDs
    filtered_paragraphs = []
    filtered_ids = []

    # Extract the text and IDs of paragraphs that match the query results
    for paragraph_id in results:
        # Join the tokens of each paragraph into a single string for processing
        filtered_paragraphs.append(' '.join(cleaned_paragraphs[paragraph_id]['tokens']))
        filtered_ids.append(paragraph_id)

    # Initialize and compute the TF-IDF matrix of the paragraphs
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(filtered_paragraphs)

    # Return the TF-IDF matrix, paragraphs, their IDs and TF-IDF vectorizer
    return tfidf_matrix, filtered_paragraphs, filtered_ids, vectorizer
    

Κατάταξη αποτελεσμάτων με τον αλγόριθμο Vector Space Model και επιστροφή της παραγράφου με μεγαλύτερη τη βαθμολόγηση από κάθε σχετικό σύνδεσμο.

In [14]:
def vector_space_model(cleaned_query, tfidf_matrix, original_paragraphs, filtered_ids, vectorizer):
    # Join the query tokens into a string
    cleaned_query = ' '.join(cleaned_query)

    # Transform the query into a TF-IDF vector using the TF-IDF vectorizer
    query_vector = vectorizer.transform([cleaned_query])

    # Compute the cosine similarity between the query and the TF-IDF matrix
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix)[0]

    # Store the paragraph with the biggest score for each link
    link_top_scores = {}

    # Look through each paragraph and its similarity score
    for paragraph_id, score in zip(filtered_ids, cosine_similarities):
        # Retrieve the original paragraph data to print
        original = original_paragraphs[paragraph_id]
        # Extract the link of current paragraph
        link = original['link']  

        # If the link is new or the score is higher than the existing one update the record
        if link not in link_top_scores or score > link_top_scores[link]['score']:
            link_top_scores[link] = {
                'paragraph_id': paragraph_id,
                'link': link,
                'text': original['text'],
                'score': score
            }

    # Sort the results in descending order
    sorted_scores = sorted(
        link_top_scores.values(),
        key = lambda x: x['score'],
        reverse = True
    )

    # Return highest ranked paragraphs 
    return sorted_scores


Κατάταξη αποτελεσμάτων με τον αλγόριθμο Okapi BM25 και επιστροφή της παραγράφου με τη μεγαλύτερη βαθμολόγηση από κάθε σχετικό σύνδεσμο.

In [15]:
def okapi_bm25(cleaned_query, filtered_paragraphs, filtered_ids, original_paragraphs):
    # Tokenize the filtered paragraphs
    tokenized_paragraphs = []

    # For every paragraph
    for paragraph in filtered_paragraphs:
        # Split into tokens
        tokens = paragraph.split(' ')
        tokenized_paragraphs.append(tokens)

    # Initialize BM25 Okapi and fit it on the tokenized paragraphs
    bm25 = BM25Okapi(tokenized_paragraphs)

    # Compute the BM25 scores for the query
    bm25_scores = bm25.get_scores(cleaned_query)

    # Keep track of the highest ranked paragraph for each link
    link_top_scores = {}

    # Look through the filtered paragraph IDs and their BM25 scores
    for paragraph_id, score in zip(filtered_ids, bm25_scores):
        # Retrieve original paragraph data for printing
        original = original_paragraphs[paragraph_id]
        link = original['link']

        # Store the paragraph only if it has the highest score for this link
        if link not in link_top_scores or score > link_top_scores[link]['score']:
            link_top_scores[link] = {
                'paragraph_id': paragraph_id,
                'link': link,
                'text': original['text'],
                'score': score
            }

    # Sort the results descending order
    sorted_scores = sorted(
        link_top_scores.values(),
        key = lambda x: x['score'],
        reverse = True
    )

    # Return highest ranked paragraphs
    return sorted_scores


Δημιουργία μηχανής αναζήτησης (Search Engine) με τη δυνατότητα επιλογής αλγόριθμου αναζήτησης και εισαγωγή ερωτήματος από τον χρήστη. 

In [16]:
def search_engine(inverted_index, original_paragraphs, cleaned_paragraphs):
    # Select the ranking algorithm
    toggle_buttons = widgets.ToggleButtons(
        options = ['Boolean retrieval', 'Vector Space Model', 'Okapi BM25'],
        description = 'Select ranking algorithm'
    )
    
    space = widgets.HTML(value = '<br>')

    # Enter search queries
    input_text = widgets.Text(
        placeholder = 'Input search query here',
        layout = widgets.Layout(width = '70%')
    )
    
    # Button to activate searching
    search_button = widgets.Button(
        description = 'Search',
        button_style = 'primary'
    )
    
    # Display search results
    output = widgets.Output()

    # Handles the search when the button is clicked
    def search(b):
        # Clear previous results
        output.clear_output()  
        # Get the selected ranking algorithm
        algorithm = toggle_buttons.value  
        # Get query that user entered
        query = input_text.value  

        # Convert the query to postfix and evaluate using the inverted index
        postfix_query = infix_to_postfix(query)
        results = evaluate_postfix(postfix_query, inverted_index, len(cleaned_paragraphs))
        
        # Apply TF-IDF transformation on the resulting paragraphs
        tfidf_matrix, filtered_paragraphs, filtered_ids, vectorizer = tf_idf(results, cleaned_paragraphs)
        # Process query for ranking
        cleaned_query = clean_text(query)  

        # Display search results based on the selected ranking algorithm
        with output:
            if filtered_paragraphs:
                print(f"Total matching paragraphs: {len(filtered_ids)}\n")

                # Display results with Boolean retrieval
                if algorithm == 'Boolean retrieval':
                    displayed_links = set()
                    for i, paragraph_id in enumerate(filtered_ids):
                        original = original_paragraphs[paragraph_id]
                        if original['link'] not in displayed_links:
                            displayed_links.add(original['link'])
                            print(f"Link: {original['link']}\n{original['text']}")
                    print(f"Total links shown: {len(displayed_links)}\n")

                # Display ranked results with VSM
                elif algorithm == 'Vector Space Model':
                    ranked_results = vector_space_model(cleaned_query, tfidf_matrix, original_paragraphs, filtered_ids, vectorizer)
                    displayed_links = set()
                    for result in ranked_results:
                        displayed_links.add(result['link'])
                        print(f"Link: {result['link']}      (Score: {result['score']:.3f})\n{result['text']}")
                    print(f"Total links shown: {len(displayed_links)}\n")
                
                # Display ranked results with okapi BM25
                elif algorithm == 'Okapi BM25':
                    ranked_results = okapi_bm25(cleaned_query, filtered_paragraphs, filtered_ids, original_paragraphs)
                    displayed_links = set()
                    for result in ranked_results:
                        displayed_links.add(result['link'])
                        print(f"Link: {result['link']}      (Score: {result['score']:.3f})\n{result['text']}")
                    print(f"Total links shown: {len(displayed_links)}\n")
                
            else:
                # If no results match the query
                print(f"No results found for '{query}' using {algorithm}.")

    # Connect the search button to the search function
    search_button.on_click(search)

    # Display widgets
    display(widgets.VBox([toggle_buttons, space, input_text, search_button, output]))

search_engine(inverted_index, original_paragraphs, cleaned_paragraphs)


# 5. Αξιολόγηση συστήματος

Εισαγωγή και επεξεργασία δεδομένων από το CISI dataset. 

In [17]:
# Read and process CISI documents
def read_documents():
    # File path to the CISI documents
    fp = '/Users/vivh/ergasia/cisi/CISI.ALL/CISI.ALL'

    with open(fp, 'r') as f:
        # To merge content across lines
        merged = ''  

        # Read file line by line and merge content while preserving field tags
        for a_line in f:
            # Identify field tags (.I, .X)
            if a_line.startswith('.'):  
                merged += '\n' + a_line.strip()
            else:
                # Add text to the merged content
                merged += ' ' + a_line.strip()  

    # Store processed documents
    documents = []  
    # Temporary storage for document text
    content = ''  
    # Temporary storage for document ID
    doc_id = '' 

    # Process content line by line
    for a_line in merged.split('\n'):
        # New document identifier
        if a_line.startswith('.I'):
            # Save previous document
            if doc_id and content:  
                documents.append({'id': doc_id, 'text': content.strip()})

            # Extract document ID
            doc_id = a_line.split(' ')[1].strip()  
            # Reset for the new document
            content = ''  
        
        # End of document identifier    
        elif a_line.startswith('.X'): 
            if doc_id and content:
                documents.append({'id': doc_id, 'text': content.strip()})
                
            doc_id = ''
            content = ''
            
        else:
            # Add the content excluding the tags
            content += a_line[3:].strip() + ' '

    # Last document in the file
    if doc_id and content:
        documents.append({'id': doc_id, 'text': content.strip()})
    
    # Save processed documents to a JSON file
    store_things(documents, 'cisi_documents.json')
    return documents


# Read and process CISI queries
def read_queries():
    fp = '/Users/vivh/ergasia/cisi/CISI.QRY'

    with open(fp, 'r') as f:
        merged = '' 

        for a_line in f:
            if a_line.startswith('.'):
                merged += '\n' + a_line.strip()
            else:
                merged += ' ' + a_line.strip() 

    queries = [] 
    content = ''  
    qry_id = '' 

    for a_line in merged.split('\n'):
        if a_line.startswith('.I'):
            if qry_id and content: 
                queries.append({'id': qry_id, 'text': content.strip()})
                
            qry_id = a_line.split(' ')[1].strip() 
            content = '' 
        
        elif a_line.startswith('.W') or a_line.startswith('.T'): 
            content += a_line.strip()[3:] + ' '

    if qry_id and content:
        queries.append({'id': qry_id, 'text': content.strip()})

    store_things(queries, 'cisi_queries.json')
    return queries


# Function to read and process CISI relevance mappings
def read_mappings():
    fp = '/Users/vivh/ergasia/cisi/CISI.REL'

    with open(fp, 'r') as f:
        # Store processed mappings
        mappings = []  

        # Read file line by line
        for a_line in f:
            # Split the line into components
            voc = a_line.strip().split()  
            # Extract query ID
            qry_id = voc[0].strip() 
            # Extract document ID
            doc_id = voc[1].strip()  

            # Append the mapping as a dictionary
            mappings.append({'query_id': qry_id, 'doc_id': doc_id})

    store_things(mappings, 'cisi_mappings.json')
    return mappings


In [18]:
documents = read_documents()

display(Markdown("How some of the CISI documents are saved: <br>"))
for paragraph in documents[:10]:
    d_id = paragraph['id']
    text = paragraph['text']

    print(f"id: {d_id}")
    print(f"text: {text}\n")

How some of the CISI documents are saved: <br>

id: 1
text: 18 Editions of the Dewey Decimal Classifications Comaromi, J.P. The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad.

id: 2
text: Use Made of Technical Libraries Slater, M. This report is an analysis of 6300 acts of use in 104 technical libraries in the United Kingdom. Library use is only one aspect of the wider pattern of information use.  Information transfer in libraries is restricted to the use of documents.  It takes no account of documents used outside the library, still less of information tra

In [19]:
queries = read_queries()  

display(Markdown("How some of the CISI queries are saved: <br>"))
for paragraph in queries[:10]:
    d_id = paragraph['id']
    text = paragraph['text']

    print(f"id: {d_id}")
    print(f"text: {text}\n")

How some of the CISI queries are saved: <br>

id: 1
text: What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?

id: 2
text: How can actually pertinent data, as opposed to references or entire articles themselves, be retrieved automatically in response to information requests?

id: 3
text: What is information science?  Give definitions where possible.

id: 4
text: Image recognition and any other methods of automatically transforming printed text into computer-ready form.

id: 5
text: What special training will ordinary researchers and businessmen need for proper information management and unobstructed use of information retrieval systems? What problems are they likely to encounter?

id: 6
text: What possibilities are there for verbal communication between computers and humans, that is, communication via the spoken word?

id: 7
text: Describe presently w

In [20]:
mappings = read_mappings()

display(Markdown("How CISI mappings are saved: <br>"))
# Convert mappings to data frame
mappings_df = pd.DataFrame(mappings)
pd.set_option('display.max_colwidth', None)
display(mappings_df)


How CISI mappings are saved: <br>

,query_id,doc_id
0,1,28
1,1,35
2,1,38
3,1,42
4,1,43
...,...,...
3109,111,422
3110,111,448
3111,111,485
3112,111,503


Υπολογισμός ακρίβειας, ανάκλησης, F1-score και μέσης ακρίβειας με αγλόριθμο αναζήτησης Okapi BM25 για τα έγγραφα και ερωτήματα του CISI dataset. 

In [21]:
def testing(documents, queries, mappings):
    # Stores cleaned documents
    cleaned_paragraphs = {}

    for doc in documents:
        doc_id = doc['id']
        # Clean and tokenize the document text, then store it
        cleaned_text = clean_text(doc['text'])  
        cleaned_paragraphs[doc_id] = {'tokens': cleaned_text}

    # Build the inverted index
    inverted_index = defaultdict(set)
    for doc_id, doc in cleaned_paragraphs.items():
        for term in doc['tokens']:  # Use 'tokens' instead of 'text'
            inverted_index[term].add(doc_id)

    # Store average precision for each query
    average_precisions = []

    for query in queries:
        query_id = query['id']
        query_text = query['text']

        # Preprocess the query text
        cleaned_query = ' '.join(clean_text(query_text))
      
        # Convert the query to postfix and evaluate using the inverted index
        postfix_query = infix_to_postfix(cleaned_query)
        results = evaluate_postfix(postfix_query, inverted_index, len(cleaned_paragraphs))

        # Apply TF-IDF transformation on the resulting paragraphs
        tfidf_matrix, filtered_paragraphs, filtered_ids, vectorizer = tf_idf(results, cleaned_paragraphs)

        if not filtered_paragraphs:  
            print(f"\nQuery ID: {query_id}")
            print(f"Query: {query_text}")
            print("No matches found.\n")
            average_precisions.append(0)
            continue

        # Use TF-IDF results as input to BM25
        tokenized_docs = []
        for doc_id in filtered_ids:
            tokenized_docs.append(cleaned_paragraphs[doc_id]['tokens'])
    
        bm25 = BM25Okapi(tokenized_docs)    
        bm25_scores = bm25.get_scores(postfix_query)

        # Rank documents based on BM25 scores
        ranked_results = sorted(
            zip(filtered_ids, bm25_scores),
            key = lambda x: x[1],
            reverse = True
        )

        # Extract retrieved document IDs with scores > 0
        retrieved_docs = []
        for doc_id, score in ranked_results:
            if score > 0:
                retrieved_docs.append(doc_id)

       # Find relevant documents for this query
        relevant_docs = set()
        for mapping in mappings:
            if mapping['query_id'] == query_id:
                relevant_docs.add(mapping['doc_id'])
        
        # Calculate precision at each relevant document's position
        true_positives = 0
        precisions = []

        for i, doc_id in enumerate(retrieved_docs, start = 1):
            if doc_id in relevant_docs:
                true_positives += 1
                precision_at_k = true_positives / i
                precisions.append(precision_at_k)

        if retrieved_docs:
            precision = true_positives / len(retrieved_docs)
        else:
            precision = 0
        
        if relevant_docs:
            ap = sum(precisions) / len(relevant_docs)
            recall = true_positives / len(relevant_docs)
        else:
            ap = 0
            
        average_precisions.append(ap)

        if precision + recall > 0:
            f1_score = (2 * precision * recall) / (precision + recall)
        else:
            f1_score = 0;

        # Print metrics for the current query
        display(Markdown("Query"))
        print(f"{query_text}")
        display(Markdown("Matching document IDs"))
        print(f"{retrieved_docs}")
        display(Markdown(f"Precision: {precision:.3f}"))
        display(Markdown(f"Recall: {recall:.3f}"))
        display(Markdown(f"F1-Score: {f1_score:.3f}<br><br>"))

    # Calculate Mean Average Precision 
    map_score = sum(average_precisions) / len(average_precisions) if average_precisions else 0
    display(Markdown(f"Mean Average Precision: {map_score:.3f}"))
    

display(Markdown("Using the CISI dataset to evaluate search engine<br>"))
testing(documents, queries, mappings)


Using the CISI dataset to evaluate search engine<br>

Query

What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?


Matching document IDs

['429', '1299', '1421', '1055', '722', '666', '76', '1090', '1281', '676', '38', '413', '64', '759', '928', '1195', '65', '212', '510', '799', '1265', '589', '541', '813', '195', '1118', '848', '1091', '1124', '1009', '154', '582', '1000', '953', '767', '576', '992', '803', '851', '831', '978', '1230', '1369', '603', '465', '1449', '276', '655', '650', '783', '711', '219', '620', '894', '1432', '869', '820', '52', '201', '338', '524', '269', '415', '483', '196', '1418', '1064', '609', '482', '886', '466', '86', '1002', '322', '192', '225', '726', '1436', '1286', '1164', '1162', '604', '757', '53', '680', '204', '150', '776', '1349', '788', '1089', '906', '221', '402', '495', '875', '215', '920', '811', '854', '193', '651', '493', '863', '921', '40', '861', '715', '775', '246', '333', '1373', '189', '1028', '1197', '1227', '1196', '1272', '865', '981', '904', '354', '614', '403', '588', '687', '551', '90', '919']


...

['853', '503', '45', '576', '1124', '596', '328', '483', '1132', '643', '487', '485', '530', '608', '1421', '769', '812', '309', '636', '1118', '71', '566', '754', '825', '714', '179', '706', '175', '966', '1215', '68', '666', '842', '84', '1202', '635', '247', '508', '1139', '17', '16', '1398', '1028', '811', '64', '1045', '495', '1109', '634', '979', '834', '135', '1131', '423', '194', '591', '178', '264', '917', '1038', '1450', '200', '625', '1328', '1176', '689', '439', '174', '887', '756', '806', '1252', '374', '27', '1042', '504', '677', '593', '484', '610', '647', '1422', '311', '1218', '208', '1012', '38', '217', '249', '897', '527', '143', '227', '410', '326', '765', '385', '528', '1358', '352', '348']


Precision: 0.000

Recall: 0.333

F1-Score: 0.000<br><br>

Mean Average Precision: 0.054